In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train_E6oV3lV.csv")

In [3]:
df.head(5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
df['word_count'] = df['tweet'].apply(lambda x: len(str(x).split(" ")))

In [5]:
df[['tweet','word_count']].head()

,tweet,word_count
0,@user when a father is dysfunctional and is s...,21
1,@user @user thanks for #lyft credit i can't us...,22
2,bihday your majesty,5
3,#model i love u take with u all the time in ...,17
4,factsguide: society now #motivation,8


In [6]:
df['char_count'] = df['tweet'].str.len()

In [7]:
#number of stop words
from nltk.corpus import stopwords

In [8]:
stop = stopwords.words('english')

In [9]:
df['numstop']=df['tweet'].apply(lambda x:len([x for x in x.split() if x in stop]))

In [10]:
df[['tweet','numstop']].head()

,tweet,numstop
0,@user when a father is dysfunctional and is s...,10
1,@user @user thanks for #lyft credit i can't us...,5
2,bihday your majesty,1
3,#model i love u take with u all the time in ...,5
4,factsguide: society now #motivation,1


In [11]:
#number of hashtags

In [12]:
df['hashtag']=df['tweet'].apply(lambda x:len([x for x in x.split() if x.startswith('#')]))

In [13]:
df[['tweet','hashtag']].head()

,tweet,hashtag
0,@user when a father is dysfunctional and is s...,1
1,@user @user thanks for #lyft credit i can't us...,3
2,bihday your majesty,0
3,#model i love u take with u all the time in ...,1
4,factsguide: society now #motivation,1


In [14]:
df['numerics'] = df['tweet'].apply(lambda x : len([x for x in x.split() if x.isdigit()]))

In [15]:
#number of uppercase
df['upper'] = df['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [16]:
#preprocessing

In [17]:
#lowercase
df['tweet'] = df['tweet'].apply(lambda x :" ".join(x.lower() for x in x.split()))

In [18]:
#removing puntuation
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')

In [19]:
df['tweet'].head()

0    user when a father is dysfunctional and is so ...
1    user user thanks for lyft credit i cant use ca...
2                                  bihday your majesty
3    model i love u take with u all the time in urð...
4                    factsguide society now motivation
Name: tweet, dtype: object

In [20]:
#removal of stopwords

In [21]:
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [22]:
#common word removal
freq = pd.Series(' '.join(df['tweet']).split()).value_counts()[:10]

In [23]:
freq = list(freq.index)
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [24]:
#rare word removal
freq = pd.Series(' '.join(df['tweet']).split()).value_counts()[-10:]

In [25]:
freq = list(freq.index)
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [28]:
from textblob import Textblob

ImportError: cannot import name 'Textblob'

In [32]:
#spell correct
from textblob import TextBlob,Word,Blobber
df['tweet'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    father dysfunctional selfish drags kiss dysfun...
1    thanks left credit can use cause dont offer wh...
2                                       midday majesty
3                               model take or ðððð ððð
4                        factsguide society motivation
Name: tweet, dtype: object

In [ ]:
#tokenization

In [34]:
TextBlob(df['tweet'][1]).words

WordList(['thanks', 'lyft', 'credit', 'cant', 'use', 'cause', 'dont', 'offer', 'wheelchair', 'vans', 'pdx', 'disapointed', 'getthanked'])

In [36]:
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['tweet'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0        father dysfunct selfish drag kid dysfunct run
1    thank lyft credit cant use caus dont offer whe...
2                                       bihday majesti
3                              model take urð ðððð ððð
4                              factsguid societi motiv
Name: tweet, dtype: object

In [38]:
from textblob import Word
df['tweet'] = df['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['tweet'].head()

0    father dysfunctional selfish drag kid dysfunct...
1    thanks lyft credit cant use cause dont offer w...
2                                       bihday majesty
3                              model take urð ðððð ððð
4                        factsguide society motivation
Name: tweet, dtype: object

In [40]:
#n-grams
TextBlob(df['tweet'][0]).ngrams(2)

[WordList(['father', 'dysfunctional']),
 WordList(['dysfunctional', 'selfish']),
 WordList(['selfish', 'drag']),
 WordList(['drag', 'kid']),
 WordList(['kid', 'dysfunction']),
 WordList(['dysfunction', 'run'])]

In [42]:
#term frequency
tf1 = (df['tweet'][1:2]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
tf1.columns = ['words','tf']
tf1

,words,tf
0,cant,1
1,use,1
2,offer,1
3,credit,1
4,cause,1
5,pdx,1
6,thanks,1
7,van,1
8,lyft,1
9,disapointed,1


In [44]:
import numpy as np

In [46]:
#IDF
for i,word in enumerate(tf1['words']):
  tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(df[df['tweet'].str.contains(word)])))

tf1

,words,tf,idf
0,cant,1,3.538194
1,use,1,3.552287
2,offer,1,6.522155
3,credit,1,7.327781
4,cause,1,5.690172
5,pdx,1,8.762865
6,thanks,1,4.597751
7,van,1,5.236505
8,lyft,1,8.762865
9,disapointed,1,10.372303


In [47]:
#tf-idf
tf1['tfidf'] = tf1['tf'] * tf1['idf']
tf1

,words,tf,idf,tfidf
0,cant,1,3.538194,3.538194
1,use,1,3.552287,3.552287
2,offer,1,6.522155,6.522155
3,credit,1,7.327781,7.327781
4,cause,1,5.690172,5.690172
5,pdx,1,8.762865,8.762865
6,thanks,1,4.597751,4.597751
7,van,1,5.236505,5.236505
8,lyft,1,8.762865,8.762865
9,disapointed,1,10.372303,10.372303


In [48]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(df['tweet'])
train_bow

<31962x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 128383 stored elements in Compressed Sparse Row format>

In [49]:
#sentiment analysis

In [51]:
df['tweet'][:5].apply(lambda x: TextBlob(x).sentiment)

0    (-0.3, 0.5354166666666667)
1                    (0.2, 0.2)
2                    (0.0, 0.0)
3                    (0.0, 0.0)
4                    (0.0, 0.0)
Name: tweet, dtype: object

In [52]:
df['sentiment'] = df['tweet'].apply(lambda x: TextBlob(x).sentiment[0] )
df[['tweet','sentiment']].head()

,tweet,sentiment
0,father dysfunctional selfish drag kid dysfunct...,-0.3
1,thanks lyft credit cant use cause dont offer w...,0.2
2,bihday majesty,0.0
3,model take urð ðððð ððð,0.0
4,factsguide society motivation,0.0


In [53]:
#word embedding
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

ModuleNotFoundError: No module named 'gensim'

In [54]:
from gensim.models import KeyedVectors # load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

ModuleNotFoundError: No module named 'gensim'